<a href="https://colab.research.google.com/github/Frans-Grau/ElectricVehicleVsChargingStations/blob/main/EVPopulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pyspark

In [3]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("EVPop").getOrCreate()

In [4]:
url = "https://raw.githubusercontent.com/Frans-Grau/ElectricVehicleVsChargingStations/main/Databases/EVPopulation.csv"
from pyspark import SparkFiles
spark.sparkContext.addFile(url)

dfEV = spark.read.csv("file://"+SparkFiles.get("EVPopulation.csv"), header=True, inferSchema= True)

In [5]:
dfEV.show(1)

+----------+------+-------+-----+-----------+----------+-------------+---------+---------------------+-------------------------------------------------+--------------+---------+--------------------+--------------+--------------------+----------------+-----------------+
|VIN (1-10)|County|   City|State|Postal Code|Model Year|         Make|    Model|Electric Vehicle Type|Clean Alternative Fuel Vehicle (CAFV) Eligibility|Electric Range|Base MSRP|Legislative District|DOL Vehicle ID|    Vehicle Location|Electric Utility|2020 Census Tract|
+----------+------+-------+-----+-----------+----------+-------------+---------+---------------------+-------------------------------------------------+--------------+---------+--------------------+--------------+--------------------+----------------+-----------------+
|WDC0G5EB7K|Louisa|Bumpass|   VA|      23024|      2019|MERCEDES-BENZ|GLC-CLASS| Plug-in Hybrid El...|                             Not eligible due ...|            10|        0|             

In [6]:
dfEV.printSchema()

root
 |-- VIN (1-10): string (nullable = true)
 |-- County: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Model Year: integer (nullable = true)
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Electric Vehicle Type: string (nullable = true)
 |-- Clean Alternative Fuel Vehicle (CAFV) Eligibility: string (nullable = true)
 |-- Electric Range: integer (nullable = true)
 |-- Base MSRP: integer (nullable = true)
 |-- Legislative District: integer (nullable = true)
 |-- DOL Vehicle ID: integer (nullable = true)
 |-- Vehicle Location: string (nullable = true)
 |-- Electric Utility: string (nullable = true)
 |-- 2020 Census Tract: long (nullable = true)



## EDA on the filtered DF -> State = Washington

In [7]:
### Filter dfEV to just values in WA and save into a new dataframe
dfWA = dfEV.filter(dfEV.State == 'WA')
dfWA.groupBy('State').count().show()

+-----+------+
|State| count|
+-----+------+
|   WA|114312|
+-----+------+



In [34]:
### Check for Null values -> the whole df
from pyspark.sql.functions import isnan, when, count, col, isnull
dfWA.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dfWA.columns]).show()

+----------+------+----+-----+-----------+----------+----+-----+---------------------+-------------------------------------------------+--------------+---------+--------------------+--------------+----------------+----------------+-----------------+
|VIN (1-10)|County|City|State|Postal Code|Model Year|Make|Model|Electric Vehicle Type|Clean Alternative Fuel Vehicle (CAFV) Eligibility|Electric Range|Base MSRP|Legislative District|DOL Vehicle ID|Vehicle Location|Electric Utility|2020 Census Tract|
+----------+------+----+-----+-----------+----------+----+-----+---------------------+-------------------------------------------------+--------------+---------+--------------------+--------------+----------------+----------------+-----------------+
|         0|     0|   0|    0|          0|         0|   0|   84|                    0|                                                0|             0|        0|                   0|             0|              27|             159|                0|


In [33]:
### Check for null values -> 1 column at a time 
from pyspark.sql.functions import col 
dfWA.filter(col('Model').isNull()).count()

84

In [16]:
### County information
dfWA.groupBy('County').count().orderBy('count', ascending=False).show()

+------------+-----+
|      County|count|
+------------+-----+
|        King|60059|
|   Snohomish|12720|
|      Pierce| 8684|
|       Clark| 6827|
|    Thurston| 4207|
|      Kitsap| 3902|
|     Whatcom| 2873|
|     Spokane| 2835|
|      Benton| 1394|
|      Island| 1307|
|      Skagit| 1269|
|     Clallam|  741|
|    San Juan|  729|
|   Jefferson|  699|
|      Chelan|  654|
|      Yakima|  633|
|     Cowlitz|  576|
|       Mason|  555|
|       Lewis|  437|
|Grays Harbor|  398|
+------------+-----+
only showing top 20 rows



In [15]:
### City Information
dfWA.groupBy('City').count().orderBy('count', ascending=False).show()

+-------------+-----+
|         City|count|
+-------------+-----+
|      Seattle|20619|
|     Bellevue| 5990|
|      Redmond| 4282|
|    Vancouver| 4108|
|     Kirkland| 3639|
|      Bothell| 3407|
|    Sammamish| 3371|
|       Renton| 2857|
|      Olympia| 2802|
|       Tacoma| 2411|
|   Bellingham| 1980|
|     Issaquah| 1829|
|      Spokane| 1697|
|Mercer Island| 1669|
|  Woodinville| 1624|
|         Kent| 1620|
|      Everett| 1520|
|     Lynnwood| 1505|
|    Snohomish| 1350|
|      Edmonds| 1303|
+-------------+-----+
only showing top 20 rows



In [11]:
### Value Counts
dfWA.groupBy('Make').count().orderBy('count', ascending=False).show()

+----------+-----+
|      Make|count|
+----------+-----+
|     TESLA|52536|
|    NISSAN|12826|
| CHEVROLET|10253|
|      FORD| 6047|
|       BMW| 4742|
|       KIA| 4553|
|    TOYOTA| 4422|
|VOLKSWAGEN| 2715|
|     VOLVO| 2528|
|      AUDI| 2368|
|  CHRYSLER| 1815|
|   HYUNDAI| 1535|
|      JEEP| 1171|
|    RIVIAN|  991|
|   PORSCHE|  820|
|      FIAT|  808|
|     HONDA|  786|
|      MINI|  671|
|  POLESTAR|  584|
|MITSUBISHI|  580|
+----------+-----+
only showing top 20 rows

